# UKZN COMP721 Project

## Authors:

Sumeith Ishwanthlal (219006284)  

Alexander Goudemond (219030365)

In this notebook, the authors will investigate how to design 1 dedicated dataset to be used for the COMP721 project.

# Current Dataset Breakdown

It is worth noting the provided datasets contains 12 documents:

- players.txt

- player_regular_season.txt

- player_regular_season_career.txt

- player_playoffs.txt

- player_playoffs_career.txt

- player_allstar.txt

- teams.txt

- team_season.txt

- draft.txt

- coaches_season.txt

- coaches_career.txt

Of those documents, this notebook will only consider the following documents:

*teams.txt, team_season.txt*


# Read in Datasets

Let us begin by reading in the 2 documents, using Pandas:

In [1]:
import pandas as pd

In [2]:
teams_df = pd.read_csv('databasebasketball/teams.txt')
team_season_df = pd.read_csv('databasebasketball/team_season.txt')

## Look at datasets

### teams_df

In [3]:
teams_df

,team,location,name,leag
0,ANA,Anaheim,Amigos,A
1,AND,Anderson,Duffey Packers,N
2,ATL,Atlanta,Hawks,N
3,BA1,Baltimore,Bullets,N
4,BAL,Baltimore,Bullets,N
...,...,...,...,...
91,WA1,Washington,Bullets,N
92,WAC,Washington,Caps,A
93,WAS,Washington,Wizards,N
94,WAT,Waterloo,Hawks,N


It looks like the 'teams_df' contains, at most, 96 unique teams. Let us verify that:

In [4]:
len(pd.unique(teams_df["team"]))

91

Okay, so there are 5 duplicate teams. Let us find them:

In [5]:
duplicates = teams_df["team"].duplicated()

for i in range(len(teams_df["team"])):
    if (duplicates[i] == True):
        print(teams_df["team"][i], "at index", i)

DEN at index 20
IND at index 30
NYN at index 57
SAS at index 71
STL at index 81


Let's now find the rows with those team names:

In [6]:
teams_df.iloc[[19, 20, 29, 30, 56, 57, 70, 71, 80, 81]]

,team,location,name,leag
19,DEN,Denver,Rockets,A
20,DEN,Denver,Nuggets,N
29,IND,Indiana,Pacers,A
30,IND,Indiana,Pacers,N
56,NYN,New York,Nets,A
57,NYN,New York,Nets,N
70,SAS,San Antonio,Spurs,A
71,SAS,San Antonio,Spurs,N
80,STL,St. Louis,Spirits,A
81,STL,St. Louis,Hawks,N


As we can see, those 5 locations have 2 names! We need to consider that when designing our own dataset

Also notice how each distinct name belongs to a different league. It cannot be the case that DEN plays against itself in League A. We can use that!

In [7]:
bools = teams_df["leag"] == "A"
teams_df_league_a = teams_df # duplicate dataframe

for i in range(len(teams_df)):
    # print(bools[i])
    if (bools[i] == False):
        teams_df_league_a = teams_df_league_a.drop(index=i)


print(len(teams_df_league_a))

teams_df_league_a


29


,team,location,name,leag
0,ANA,Anaheim,Amigos,A
8,CAR,Carolina,Cougars,A
19,DEN,Denver,Rockets,A
22,DLC,Dallas,Chaparrals,A
24,FLA,Floridians,NaN,A
27,HMV,Houston,Mavericks,A
29,IND,Indiana,Pacers,A
34,KEN,Kentucky,Colonels,A
37,LAS,Los Angeles,Stars,A
39,MFL,Miami,Floridians,A


In [8]:
bools = teams_df["leag"] == "N"
teams_df_league_n = teams_df # duplicate dataframe

for i in range(len(teams_df)):
    # print(bools[i])
    if (bools[i] == False):
        teams_df_league_n = teams_df_league_n.drop(index=i)


print(len(teams_df_league_n))

teams_df_league_n


67


,team,location,name,leag
1,AND,Anderson,Duffey Packers,N
2,ATL,Atlanta,Hawks,N
3,BA1,Baltimore,Bullets,N
4,BAL,Baltimore,Bullets,N
5,BOS,Boston,Celtics,N
...,...,...,...,...
89,VAN,Vancouver,Grizzlies,N
91,WA1,Washington,Bullets,N
93,WAS,Washington,Wizards,N
94,WAT,Waterloo,Hawks,N


In [9]:
# verify data is preserved:

len(teams_df_league_n) + len(teams_df_league_a) == len(teams_df)

True

### team_season_df

Similar to teams_df, we must isolate the leagues and determine the unique teams:

In [10]:
team_season_df

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
0,BOS,1946,N,1397,5133,811,1375,0,0,0,...,0,0,0,0,0,0,3900,0.000000,22,38
1,CH1,1946,N,1879,6309,939,1550,0,0,0,...,0,0,0,0,0,0,4471,0.000000,39,22
2,CL1,1946,N,1674,5699,903,1428,0,0,0,...,0,0,0,0,0,0,4308,0.000000,30,30
3,DE1,1946,N,1437,5843,923,1494,0,0,0,...,0,0,0,0,0,0,3918,0.000000,20,40
4,NYK,1946,N,1465,5255,951,1438,0,0,0,...,0,0,0,0,0,0,3840,0.000000,33,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,SAS,2004,N,2923,6450,1535,2120,987,2489,3476,...,1891,594,1246,421,323,881,7248,91.455696,59,23
1183,SEA,2004,N,2882,6498,1683,2131,1041,2311,3352,...,1948,508,1127,390,470,1317,7925,91.244949,52,30
1184,TOR,2004,N,2952,6656,1626,2101,844,2444,3288,...,1795,507,1188,360,435,1222,8311,95.012611,33,49
1185,UTA,2004,N,2828,6301,1719,2272,1047,2243,3290,...,2057,646,1213,474,486,1297,7975,91.195145,26,56


How many unique teams are there?

In [11]:
unique_teams = team_season_df["team"].unique()

len(unique_teams)

91

Extract teams by league:

In [12]:
bools = team_season_df["leag"] == "A"
team_season_df_league_a = team_season_df # duplicate dataframe

for i in range(len(team_season_df)):
    # print(bools[i])
    if (bools[i] == False):
        team_season_df_league_a = team_season_df_league_a.drop(index=i)


print(len(team_season_df_league_a))

team_season_df_league_a


94


,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
199,ANA,1967,A,3172,7606,2141,2916,0,0,4158,...,1977,0,0,0,99,0,9057,0.0,25,53
204,DEN,1967,A,3119,7271,1981,2725,0,0,4121,...,1903,0,0,0,116,0,7914,0.0,45,33
206,DLC,1967,A,3260,7167,2027,2810,0,0,4030,...,1887,0,0,0,133,0,8470,0.0,46,32
207,HMV,1967,A,3094,7731,1774,2537,0,0,4207,...,1839,0,0,0,105,0,8407,0.0,29,49
208,IND,1967,A,3174,7397,2102,2971,0,0,4257,...,2013,0,0,0,141,0,8530,0.0,38,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,SAS,1975,A,3888,7945,1905,2342,0,0,4232,...,2141,0,0,0,122,0,9362,0.0,50,34
431,SDS,1975,A,432,1020,217,278,0,0,550,...,273,0,0,0,12,0,1138,0.0,3,8
433,STL,1975,A,3790,8177,1500,1985,0,0,4288,...,1888,0,0,0,89,0,9416,0.0,35,49
434,UTS,1975,A,715,1498,386,467,0,0,781,...,461,0,0,0,20,0,1866,0.0,4,12


In [13]:
bools = team_season_df["leag"] == "N"
team_season_df_league_n = team_season_df # duplicate dataframe

for i in range(len(team_season_df)):
    # print(bools[i])
    if (bools[i] == False):
        team_season_df_league_n = team_season_df_league_n.drop(index=i)


print(len(team_season_df_league_n))

team_season_df_league_n


1093


,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
0,BOS,1946,N,1397,5133,811,1375,0,0,0,...,0,0,0,0,0,0,3900,0.000000,22,38
1,CH1,1946,N,1879,6309,939,1550,0,0,0,...,0,0,0,0,0,0,4471,0.000000,39,22
2,CL1,1946,N,1674,5699,903,1428,0,0,0,...,0,0,0,0,0,0,4308,0.000000,30,30
3,DE1,1946,N,1437,5843,923,1494,0,0,0,...,0,0,0,0,0,0,3918,0.000000,20,40
4,NYK,1946,N,1465,5255,951,1438,0,0,0,...,0,0,0,0,0,0,3840,0.000000,33,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,SAS,2004,N,2923,6450,1535,2120,987,2489,3476,...,1891,594,1246,421,323,881,7248,91.455696,59,23
1183,SEA,2004,N,2882,6498,1683,2131,1041,2311,3352,...,1948,508,1127,390,470,1317,7925,91.244949,52,30
1184,TOR,2004,N,2952,6656,1626,2101,844,2444,3288,...,1795,507,1188,360,435,1222,8311,95.012611,33,49
1185,UTA,2004,N,2828,6301,1719,2272,1047,2243,3290,...,2057,646,1213,474,486,1297,7975,91.195145,26,56


In [14]:
# verify data is preserved:

len(team_season_df_league_a) + len(team_season_df_league_n) == len(team_season_df)

True

Now, per league, how many distinct teams are there?

In [15]:
len(team_season_df_league_a["team"].unique())

29

In [16]:
len(team_season_df_league_n["team"].unique())

67

Notice how these teams account for all our data in the teams.csv file:

In [17]:
len(team_season_df_league_a["team"].unique()) + len(team_season_df_league_n["team"].unique()) == len(teams_df)

True

Now that we have isolated 2 sets of data - we can use them to generate unique team datasets for our models

# Determining a suitable benchmark

The following attributes are present in the team_season.txt file:

In [18]:
print(len(team_season_df.columns.values))
print(team_season_df.columns.values)

36
['team' 'year' 'leag' 'o_fgm' 'o_fga' 'o_ftm' 'o_fta' 'o_oreb' 'o_dreb'
 'o_reb' 'o_asts' 'o_pf' 'o_stl' 'o_to' 'o_blk' 'o_3pm' 'o_3pa' 'o_pts'
 'd_fgm' 'd_fga' 'd_ftm' 'd_fta' 'd_oreb' 'd_dreb' 'd_reb' 'd_asts' 'd_pf'
 'd_stl' 'd_to' 'd_blk' 'd_3pm' 'd_3pa' 'd_pts' 'pace' 'won' 'lost']


Notice how some of those datasets contain a 'o' infront of them, and similar attributes contain a 'd' infront of them. This is by design - the 'o' stands for **offensive play**, whereas the 'd' stands for **defensive play**

In this section of the notebook, we will isolate the 30 attributes containing either a 'o' or a 'd', and use them to generate some benchmarks.

The assumption the authors make is that the statistics for a team over a season contributes to the probability that that team will win. To generate a number that can be used as a probability - this notebook needs to collect all the data a team has generated over a season (specifically for 2004), and then add the offensive attributes together as well as the defensive attributes together. The only attribute that should be negative is the attribute representing Player Fouls - which is understood to be a negative play in a team.

To visualize these scores, the following may assist:

Let the *offensive_set* = {'o_fgm' 'o_fga' 'o_ftm' 'o_fta' 'o_oreb' 'o_dreb', 'o_reb' 'o_asts' 'o_pf' 'o_stl' 'o_to' 'o_blk' 'o_3pm' 'o_3pa' 'o_pts'} and ...

... the *defensive_set* = {'d_fgm' 'd_fga' 'd_ftm' 'd_fta' 'd_oreb' 'd_dreb', 'd_reb' 'd_asts' 'd_pf' 'd_stl' 'd_to' 'd_blk' 'd_3pm' 'd_3pa' 'd_pts'}. 

Then:

*offensize_score* = $-$ 'o_pf' $+ \sum $ *offensive_set* / {'o_pf'}

*defensize_score* = $-$ 'd_pf' $+ \sum $ *deffensive_set* / {'d_pf'} , where 'pf' represents the personal fouls

team_season_df_league_a and team_season_df_league_n may now be prepared:

### Drop all rows except 2014

In [19]:
team_season_df_league_a

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
199,ANA,1967,A,3172,7606,2141,2916,0,0,4158,...,1977,0,0,0,99,0,9057,0.0,25,53
204,DEN,1967,A,3119,7271,1981,2725,0,0,4121,...,1903,0,0,0,116,0,7914,0.0,45,33
206,DLC,1967,A,3260,7167,2027,2810,0,0,4030,...,1887,0,0,0,133,0,8470,0.0,46,32
207,HMV,1967,A,3094,7731,1774,2537,0,0,4207,...,1839,0,0,0,105,0,8407,0.0,29,49
208,IND,1967,A,3174,7397,2102,2971,0,0,4257,...,2013,0,0,0,141,0,8530,0.0,38,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,SAS,1975,A,3888,7945,1905,2342,0,0,4232,...,2141,0,0,0,122,0,9362,0.0,50,34
431,SDS,1975,A,432,1020,217,278,0,0,550,...,273,0,0,0,12,0,1138,0.0,3,8
433,STL,1975,A,3790,8177,1500,1985,0,0,4288,...,1888,0,0,0,89,0,9416,0.0,35,49
434,UTS,1975,A,715,1498,386,467,0,0,781,...,461,0,0,0,20,0,1866,0.0,4,12


In [20]:
team_season_df_league_a["year"].iloc[0]

1967

In [21]:
# where does 2004 begin?
for i in range(len(team_season_df_league_a)):
    if (team_season_df_league_a["year"].iloc[i] == 2004):
        print("First index appears at position", i)
        break

In [22]:
# where does 2004 begin?
for i in range(len(team_season_df_league_n)):
    if (team_season_df_league_n["year"].iloc[i] == 2004):
        print("First index appears at position", i)
        break

First index appears at position 1063


Apparently league a does not contain the year 2004! Thus, we can ignore it

In [23]:
team_season_df_league_n = team_season_df_league_n.iloc[1063:]

team_season_df_league_n

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
1157,ATL,2004,N,2942,6672,1417,1994,1100,2335,3435,...,1777,716,1235,484,467,1232,8401,93.948730,13,69
1158,BOS,2004,N,3046,6511,1775,2323,909,2438,3347,...,1937,714,1248,402,488,1370,8233,95.735855,45,37
1159,CHI,2004,N,2849,6592,1536,2048,1001,2591,3592,...,1914,685,1272,434,416,1245,7658,96.283737,47,35
1160,CHR,2004,N,2961,6859,1487,2096,1083,2335,3418,...,1857,624,1304,446,442,1224,8220,96.201630,18,64
1161,CLE,2004,N,2990,6687,1634,2174,1117,2352,3469,...,1770,549,1226,419,422,1122,7849,93.840187,42,40
1162,DAL,2004,N,3058,6691,1826,2314,990,2530,3520,...,1898,584,1258,399,451,1366,7934,95.204948,58,24
1163,DEN,2004,N,3038,6615,1765,2313,967,2473,3440,...,1960,636,1325,458,463,1342,7995,96.029640,49,33
1164,DET,2004,N,2851,6421,1588,2150,1054,2507,3561,...,1851,610,1138,367,410,1213,7336,89.839737,54,28
1165,GSW,2004,N,3029,7039,1412,1955,1069,2436,3505,...,1782,570,1231,417,450,1240,8271,95.689407,34,48
1166,HOU,2004,N,2846,6419,1551,1986,874,2601,3475,...,1773,607,1091,335,454,1343,7465,91.303230,51,31


Now, after all that processing, how many distinct classes do we have?

In [24]:
len(team_season_df_league_n["team"].unique())

30

only 30 teams to work with!

### Benchmark team_season_df_league_n

In [25]:
team_season_df_league_n.drop(["year", "leag"], axis = 1, inplace = True) 

C:\Users\G5\AppData\Local\Temp\ipykernel_14424\1173872428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_season_df_league_n.drop(["year", "leag"], axis = 1, inplace = True)


In [26]:
team_season_df_league_n

,team,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
1157,ATL,2942,6672,1417,1994,1100,2335,3435,1614,2009,...,1777,716,1235,484,467,1232,8401,93.948730,13,69
1158,BOS,3046,6511,1775,2323,909,2438,3347,1810,2000,...,1937,714,1248,402,488,1370,8233,95.735855,45,37
1159,CHI,2849,6592,1536,2048,1001,2591,3592,1743,2028,...,1914,685,1272,434,416,1245,7658,96.283737,47,35
1160,CHR,2961,6859,1487,2096,1083,2335,3418,1794,1893,...,1857,624,1304,446,442,1224,8220,96.201630,18,64
1161,CLE,2990,6687,1634,2174,1117,2352,3469,1851,1850,...,1770,549,1226,419,422,1122,7849,93.840187,42,40
1162,DAL,3058,6691,1826,2314,990,2530,3520,1610,1827,...,1898,584,1258,399,451,1366,7934,95.204948,58,24
1163,DEN,3038,6615,1765,2313,967,2473,3440,1958,1878,...,1960,636,1325,458,463,1342,7995,96.029640,49,33
1164,DET,2851,6421,1588,2150,1054,2507,3561,1787,1638,...,1851,610,1138,367,410,1213,7336,89.839737,54,28
1165,GSW,3029,7039,1412,1955,1069,2436,3505,1811,1833,...,1782,570,1231,417,450,1240,8271,95.689407,34,48
1166,HOU,2846,6419,1551,1986,874,2601,3475,1733,1806,...,1773,607,1091,335,454,1343,7465,91.303230,51,31


Calculate offensize and defensive scores, then ratios:

In [27]:
team_season_df_league_n["offensive"] =  - team_season_df_league_n["o_pf"]   + \
                                        team_season_df_league_n["o_3pa"]    + \
                                        team_season_df_league_n["o_3pm"]    + \
                                        team_season_df_league_n["o_asts"]   + \
                                        team_season_df_league_n["o_blk"]    + \
                                        team_season_df_league_n["o_dreb"]   + \
                                        team_season_df_league_n["o_fga"]    + \
                                        team_season_df_league_n["o_fgm"]    + \
                                        team_season_df_league_n["o_fta"]    + \
                                        team_season_df_league_n["o_ftm"]    + \
                                        team_season_df_league_n["o_oreb"]   + \
                                        team_season_df_league_n["o_pts"]    + \
                                        team_season_df_league_n["o_reb"]    + \
                                        team_season_df_league_n["o_stl"]    + \
                                        team_season_df_league_n["o_to"]

team_season_df_league_n["defensive"] =  - team_season_df_league_n["d_pf"]   + \
                                        team_season_df_league_n["d_3pa"]    + \
                                        team_season_df_league_n["d_3pm"]    + \
                                        team_season_df_league_n["d_asts"]   + \
                                        team_season_df_league_n["d_blk"]    + \
                                        team_season_df_league_n["d_dreb"]   + \
                                        team_season_df_league_n["d_fga"]    + \
                                        team_season_df_league_n["d_fgm"]    + \
                                        team_season_df_league_n["d_fta"]    + \
                                        team_season_df_league_n["d_ftm"]    + \
                                        team_season_df_league_n["d_oreb"]   + \
                                        team_season_df_league_n["d_pts"]    + \
                                        team_season_df_league_n["d_reb"]    + \
                                        team_season_df_league_n["d_stl"]    + \
                                        team_season_df_league_n["d_to"]    

C:\Users\G5\AppData\Local\Temp\ipykernel_14424\611825272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_season_df_league_n["offensive"] =  - team_season_df_league_n["o_pf"]   + \
C:\Users\G5\AppData\Local\Temp\ipykernel_14424\611825272.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_season_df_league_n["defensive"] =  - team_season_df_league_n["d_pf"]   + \


In [28]:
team_season_df_league_n["ratio_offensive"] = team_season_df_league_n["offensive"] / (team_season_df_league_n["offensive"] + team_season_df_league_n["defensive"])
team_season_df_league_n["ratio_defensive"] = team_season_df_league_n["defensive"] / (team_season_df_league_n["offensive"] + team_season_df_league_n["defensive"])

C:\Users\G5\AppData\Local\Temp\ipykernel_14424\3943748517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_season_df_league_n["ratio_offensive"] = team_season_df_league_n["offensive"] / (team_season_df_league_n["offensive"] + team_season_df_league_n["defensive"])
C:\Users\G5\AppData\Local\Temp\ipykernel_14424\3943748517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_season_df_league_n["ratio_defensive"] = team_season_df_league_n["defensive"] / (team_season_df_league_n["offensive"] + team_s

Check the dataframe:

In [29]:
team_season_df_league_n

,team,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,...,d_3pm,d_3pa,d_pts,pace,won,lost,offensive,defensive,ratio_offensive,ratio_defensive
1157,ATL,2942,6672,1417,1994,1100,2335,3435,1614,2009,...,467,1232,8401,93.948730,13,69,30674,33006,0.481690,0.518310
1158,BOS,3046,6511,1775,2323,909,2438,3347,1810,2000,...,488,1370,8233,95.735855,45,37,32539,33181,0.495116,0.504884
1159,CHI,2849,6592,1536,2048,1001,2591,3592,1743,2028,...,416,1245,7658,96.283737,47,35,31929,31849,0.500627,0.499373
1160,CHR,2961,6859,1487,2096,1083,2335,3418,1794,1893,...,442,1224,8220,96.201630,18,64,31397,32810,0.488997,0.511003
1161,CLE,2990,6687,1634,2174,1117,2352,3469,1851,1850,...,422,1122,7849,93.840187,42,40,31798,31171,0.504979,0.495021
1162,DAL,3058,6691,1826,2314,990,2530,3520,1610,1827,...,451,1366,7934,95.204948,58,24,33124,32263,0.506584,0.493416
1163,DEN,3038,6615,1765,2313,967,2473,3440,1958,1878,...,463,1342,7995,96.029640,49,33,32575,32246,0.502538,0.497462
1164,DET,2851,6421,1588,2150,1054,2507,3561,1787,1638,...,410,1213,7336,89.839737,54,28,31556,29760,0.514645,0.485355
1165,GSW,3029,7039,1412,1955,1069,2436,3505,1811,1833,...,450,1240,8271,95.689407,34,48,33089,33524,0.496735,0.503265
1166,HOU,2846,6419,1551,1986,874,2601,3475,1733,1806,...,454,1343,7465,91.303230,51,31,31625,30720,0.507258,0.492742


What about quartiles?

In [32]:
quantile_league_n = team_season_df_league_n.quantile([0, 0.25, .5, 0.75, 1], interpolation="nearest")

quantile_league_n

,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,...,d_3pm,d_3pa,d_pts,pace,won,lost,offensive,defensive,ratio_offensive,ratio_defensive
0.00,2668,6169,1401,1820,844,2243,3194,1487,1563,460,...,323,881,7248,89.839737,13,20,30005,29326,0.481690,0.477366
0.25,2851,6421,1535,2024,909,2374,3352,1670,1797,563,...,440,1232,7792,91.455696,33,32,31348,31164,0.493555,0.495021
0.50,2946,6592,1588,2120,985,2438,3440,1733,1850,615,...,463,1291,7949,93.840187,43,38,31798,32152,0.500359,0.499373
0.75,3034,6687,1731,2272,1041,2530,3520,1811,1937,667,...,485,1346,8268,96.008492,50,49,32766,33006,0.504979,0.506445
1.00,3351,7039,1826,2476,1133,2652,3619,2005,2189,756,...,565,1560,8470,98.476440,62,69,35633,33982,0.522634,0.518310


Okay! We can see the following information from that quartile information:

| Ratio           | Lowest Score | Highest Score |
|-----------------|--------------|---------------|
| Offensive Ratio | 0.481690     | 0.522634      |
| Defensive Ratio | 0.477366     | 0.518310      |

The benchmark we will use is the following:

Between 2 teams, team A and team B: 

**If:** team A has both the *Offensive Ratio* and *Defensive Ratio* higher than team B, **Then:** team A wins

**Else If:** team B has both the *Offensive Ratio* and *Defensive Ratio* higher than team B, **Then:** team B wins

**Else If:** team A has the *Offensive Ratio* higher than team B, **Then:** team A wins

**Else:** team B wins

That benchmark will be used to generate a unique dataset for us to use

# Unique Dataset creation

In this notebook, we will generate a unique dataset to be used to detect the outcome of a model, given information on 2 teams.

The dataset will be constructed in the following way:

| team_a | pace | won | lost | cumulative_score | team_b | pace | won | lost | cumulative_score| winner |
|--------|------|-----|------|------------------|--------|------|-----|------|-----------------|--------|
|        |      |     |      |                  |        |      |     |      |                 |        |

The winner attribute will be identified by comparing the benchmarks generated above. Each row will represent the match between 2 teams, and will be generated using the data in quantile_league_a.

Recall the number of distinct teams:

In [31]:
len(team_season_df_league_n["team"].unique())

30

In [40]:
# team_season_df_league_n.iloc[0]

In [53]:
unique_teams = len(team_season_df_league_n["team"].unique())

new_df = []

for i in range(unique_teams):
    for j in range(unique_teams):
        # skip same team match up
        if (team_season_df_league_n["team"].iloc[i] == team_season_df_league_n["team"].iloc[j]):
            continue

        #### calculate winner
        # team a wins
        if (    (team_season_df_league_n["offensive"].iloc[i] > team_season_df_league_n["offensive"].iloc[j])
            and
                (team_season_df_league_n["defensive"].iloc[i] > team_season_df_league_n["defensive"].iloc[j])
            ):
            winner = 1
        # team b wins
        elif    (   (team_season_df_league_n["offensive"].iloc[j] > team_season_df_league_n["offensive"].iloc[i])
                and
                    (team_season_df_league_n["defensive"].iloc[j] > team_season_df_league_n["defensive"].iloc[i])
                ):
            winner = 0
        # team a wins
        if  (    (team_season_df_league_n["offensive"].iloc[i] > team_season_df_league_n["offensive"].iloc[j])
            ):
            winner = 1
        # team b wins
        else:
            winner = 0

        # append entry
        temp = (
            team_season_df_league_n["team"].iloc[i],
            team_season_df_league_n["pace"].iloc[i],
            team_season_df_league_n["won"].iloc[i],
            team_season_df_league_n["lost"].iloc[i],
            team_season_df_league_n["offensive"].iloc[i] + team_season_df_league_n["defensive"].iloc[i],
            team_season_df_league_n["team"].iloc[j],
            team_season_df_league_n["pace"].iloc[j],
            team_season_df_league_n["won"].iloc[j],
            team_season_df_league_n["lost"].iloc[j],
            team_season_df_league_n["offensive"].iloc[j] + team_season_df_league_n["defensive"].iloc[j],
            winner
        )

        new_df.append(temp)
  

new_df = pd.DataFrame(new_df, columns=("team_a", "pace", "won", "lost", "cumulative_score", "team_b", "pace", "won", "lost", "cumulative_score", "winner"))

new_df

,team_a,pace,won,lost,cumulative_score,team_b,pace,won,lost,cumulative_score,winner
0,ATL,93.948730,13,69,63680,BOS,95.735855,45,37,65720,0
1,ATL,93.948730,13,69,63680,CHI,96.283737,47,35,63778,0
2,ATL,93.948730,13,69,63680,CHR,96.201630,18,64,64207,0
3,ATL,93.948730,13,69,63680,CLE,93.840187,42,40,62969,0
4,ATL,93.948730,13,69,63680,DAL,95.204948,58,24,65387,0
...,...,...,...,...,...,...,...,...,...,...,...
865,WAS,96.008492,45,37,66161,SAC,96.841805,50,32,66721,0
866,WAS,96.008492,45,37,66161,SAS,91.455696,59,23,61433,1
867,WAS,96.008492,45,37,66161,SEA,91.244949,52,30,62773,1
868,WAS,96.008492,45,37,66161,TOR,95.012611,33,49,65318,1


And now, save as csv:

In [54]:
new_df.to_csv("team_success.csv", index=False) # don't include indices

This means that our dataset will contain 30 * 29 combinations == 870 rows of data

# Conclusion

We now have a dataset containing the team_success from the basketball stats! 